In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline

In [2]:
df = pd.read_stata('usa_00021.dta')

In [3]:
df['agg educd 0'] = np.where(df['educd'] >= '4 years of college', 'BA or More', np.nan)
df['agg educd 1'] = np.where(((df['educd'] >= 'high school graduate or ged') & (df['educd'] <= 'ged or alternative credential')), 'HS', df['agg educd 0'])
df['agg educd 2'] = np.where(df['educd'] < 'high school graduate or ged', 'Less than HS', df['agg educd 1'])
df['agg educd 3'] = np.where(((df['educd'] > 'ged or alternative credential') & (df['educd'] < '4 years of college')), 'Some College', df['agg educd 2']) 

df['agg educd'] = df['agg educd 3'].copy()

df.drop(columns = ['agg educd 0', 'agg educd 1', 'agg educd 2', 'agg educd 3'], inplace = True)

In [7]:
immig_educ = df.groupby(['year', 'bpld', 'agg educd',])['perwt'].sum().to_frame()
immig = df.groupby(['year', 'bpld',])['perwt'].sum().to_frame()
immig_educ.to_csv('immigrants education.csv')
immig.to_csv('immigrants no education.csv')

In [5]:
immig_educ = df.groupby(['year', 'bpld', 'agg educd',])['perwt'].sum().to_frame()
data = immig_educ.reset_index()
def make_altair_chart(country):
    brush = alt.selection(type='interval', encodings=['x'])

    upper1 = alt.Chart(data[data['bpld'] == country]).mark_line().encode(
        x = alt.X('year:T', timeUnit='year', scale={'domain': brush.ref()}),
        y = 'perwt:Q',
        color = 'agg educd',
        order = alt.Order('agg educd:O'),
    ).properties(
        title = f'{country}'.capitalize(),
        width = 800,
        height = 400,
    )

    lower1 = upper1.properties(
        selection=brush,
        height=60
    )
    chart = alt.vconcat(upper1, lower1)
    return chart

In [6]:
interact(make_altair_chart, country = df['bpld'].unique())

interactive(children=(Dropdown(description='country', options=('vietnam', 'venezuela', 'mexico', 'australia', …

<function __main__.make_altair_chart>

In [23]:
immig_educ = df.groupby(['year', 'bpld', 'agg educd',])['perwt'].sum().to_frame()
data = immig_educ.reset_index()
data = data[data['agg educd'] == 'BA or More']
data = data[(data['bpld'] == 'mexico') | (data['bpld'] == 'vietnam') | (data['bpld'] == 'india') | (data['bpld'] == 'china') | (data['bpld'] == 'philippines')]

brush = alt.selection(type='interval', encodings=['x'])

upper1 = alt.Chart(data).mark_line().encode(
    x = alt.X('year:T', timeUnit='year', scale={'domain': brush.ref()}, axis = alt.Axis(title = 'Year')),
    y = alt.Y('perwt:Q', axis = alt.Axis(title = 'Number of Adult (18-65) Immigrants arriving within the last 5 years')),
    color = 'bpld',
    order = alt.Order('bpld:O'),
).properties(
    #title = f'{country}'.capitalize(),
    width = 800,
    height = 400,
)

lower1 = upper1.properties(
    selection=brush,
    height=60
)
chart = alt.vconcat(upper1, lower1)
chart

<VegaLite 2 object>

In [38]:
immig.xs('2016', level = 0).sort_values(by = 'perwt', ascending = False).head(0)

,perwt
bpld,
china,146135.0
mexico,145667.0
india,117132.0
philippines,60819.0
vietnam,41446.0
el salvador,33049.0
iran,27753.0
korea,24582.0
guatemala,18431.0


In [36]:
511199/930789

0.549210401068341